In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn import metrics
import transformers
import time
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import warnings
warnings.filterwarnings('ignore')
model_save='pretrain6'
if not os.path.exists(model_save):
    os.mkdir(model_save)

In [ ]:
import pandas as pd
train_df=pd.read_csv('dataset/track1_round1_train_20210222.csv',header=None)
test_df=pd.read_csv('dataset/track1_round1_testA_20210222.csv',header=None) 
train_df.columns=['report_ID','description','label']
test_df.columns=['report_ID','description']
train_df.drop(['report_ID'],axis=1,inplace=True)
test_df.drop(['report_ID'],axis=1,inplace=True)
new_des=[i.strip('|').strip() for i in train_df['description'].values]
new_label=[i.strip('|').strip() for i in train_df['label'].values]
train_df['description']=new_des
train_df['label']=new_label
new_des=[i.strip('|').strip() for i in test_df['description'].values]
test_df['description']=new_des

In [ ]:
from transformers import BertTokenizerFast,BertForMaskedLM,BertConfig,\
LineByLineTextDataset,DataCollatorForLanguageModeling,Trainer,TrainingArguments
from tokenizers import BertWordPieceTokenizer

In [ ]:
# tokenizer=BertPreTokenizer()
# tokenizer.train()

In [ ]:
tokenizer=BertTokenizerFast.from_pretrained('pre/',max_len=128)

In [ ]:
config=BertConfig(vocab_size=863,
                  num_attention_heads=12,
                  num_hidden_layers=12,
                  hidden_size=768)
model=BertForMaskedLM(config=config)
print(model.num_parameters())

In [ ]:
dataset=LineByLineTextDataset(tokenizer=tokenizer,
                              file_path='pre/sentence.txt',
                              block_size=128)
data_collator =DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True,mlm_probability  = 0.15)

In [ ]:
training_args=TrainingArguments(
                output_dir=model_save,
                overwrite_output_dir=True,
                num_train_epochs=128,
                per_gpu_train_batch_size=64,
                save_total_limit=2,
                weight_decay=0.001,
                            )
trainer=Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True
    

)
trainer.train()

In [ ]:
trainer.save_model(model_save)

In [ ]:
import transformers
print(transformers.__version__)